# Task 1: Exploratory Data Analysis (EDA)

**Financial News Sentiment Analysis - Week 1 Challenge**

This notebook performs comprehensive EDA on the financial news dataset to understand:
- Descriptive statistics (headline length, articles per publisher, publication dates)
- Text analysis and topic modeling (NLP keywords/phrases)
- Time series analysis (publication frequency over time)
- Publisher analysis (most active publishers, domain analysis)


## 1. Setup and Data Loading


In [ ]:
# Standard library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from datetime import datetime
import re

# NLP imports
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download required NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords', quiet=True)

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet', quiet=True)

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
warnings.filterwarnings('ignore')

# Set paths
PROJECT_ROOT = Path('..')
DATA_DIR = PROJECT_ROOT / 'data'
FIGURES_DIR = PROJECT_ROOT / 'figures'

# Create directories if they don't exist
FIGURES_DIR.mkdir(exist_ok=True)

print("Setup complete!")


In [ ]:
# Load the financial news dataset
# Note: Update the filename based on your actual data file
data_files = list(DATA_DIR.glob('*.csv')) + list(DATA_DIR.glob('*.json'))

if not data_files:
    print("⚠️  No data files found in data/ directory.")
    print("Please ensure your dataset is placed in the data/ directory.")
    print("Expected columns: headline, url, publisher, date, stock")
else:
    # Try to load the first data file found
    data_file = data_files[0]
    print(f"Loading data from: {data_file.name}")
    
    if data_file.suffix == '.csv':
        df = pd.read_csv(data_file, parse_dates=['date'], low_memory=False)
    elif data_file.suffix == '.json':
        df = pd.read_json(data_file, convert_dates=['date'])
    
    print(f"\n✅ Data loaded successfully!")
    print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"\nFirst few rows:")
    display(df.head())
    print(f"\nColumn names: {list(df.columns)}")
    print(f"\nData types:")
    display(df.dtypes)
    print(f"\nBasic info:")
    display(df.info())


## 2. Data Cleaning and Preparation


In [ ]:
# Check for missing values
print("Missing values per column:")
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})
display(missing_df[missing_df['Missing Count'] > 0])

# Handle missing values if needed
# Drop rows with missing headlines (critical for analysis)
initial_count = len(df)
df = df.dropna(subset=['headline'])
print(f"\nDropped {initial_count - len(df)} rows with missing headlines")
print(f"Remaining rows: {len(df):,}")


In [ ]:
# Ensure date column is datetime
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'], errors='coerce', utc=True)
    # Convert from UTC-4 to UTC if needed (adjust based on your data)
    # df['date'] = df['date'] + pd.Timedelta(hours=4)
    
    # Extract date components
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['day_of_week'] = df['date'].dt.day_name()
    df['hour'] = df['date'].dt.hour
    
    print("Date column processed successfully!")
    print(f"Date range: {df['date'].min()} to {df['date'].max()}")

# Clean headline text
if 'headline' in df.columns:
    df['headline_length'] = df['headline'].str.len()
    df['headline_word_count'] = df['headline'].str.split().str.len()
    print("\nHeadline statistics calculated!")


## 3. Descriptive Statistics

### 3.1 Headline Length Statistics


In [ ]:
# Basic statistics for headline lengths
print("=== Headline Length Statistics ===")
headline_stats = df['headline_length'].describe()
display(headline_stats)

print("\n=== Headline Word Count Statistics ===")
word_count_stats = df['headline_word_count'].describe()
display(word_count_stats)

# Visualizations
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Headline length distribution
axes[0].hist(df['headline_length'], bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(df['headline_length'].mean(), color='red', linestyle='--', 
                label=f'Mean: {df["headline_length"].mean():.1f}')
axes[0].axvline(df['headline_length'].median(), color='green', linestyle='--', 
                label=f'Median: {df["headline_length"].median():.1f}')
axes[0].set_xlabel('Headline Length (characters)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Headline Lengths')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Word count distribution
axes[1].hist(df['headline_word_count'], bins=30, edgecolor='black', alpha=0.7, color='orange')
axes[1].axvline(df['headline_word_count'].mean(), color='red', linestyle='--', 
                label=f'Mean: {df["headline_word_count"].mean():.1f}')
axes[1].axvline(df['headline_word_count'].median(), color='green', linestyle='--', 
                label=f'Median: {df["headline_word_count"].median():.1f}')
axes[1].set_xlabel('Word Count')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Headline Word Counts')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'headline_length_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Headline length analysis complete!")


In [ ]:
# Count articles per publisher
publisher_counts = df['publisher'].value_counts()
publisher_stats = pd.DataFrame({
    'Article Count': publisher_counts,
    'Percentage': (publisher_counts / len(df) * 100).round(2)
})

print(f"=== Publisher Statistics ===")
print(f"Total unique publishers: {df['publisher'].nunique()}")
print(f"\nTop 20 Publishers:")
display(publisher_stats.head(20))

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Top 15 publishers bar chart
top_publishers = publisher_counts.head(15)
axes[0].barh(range(len(top_publishers)), top_publishers.values, color='steelblue')
axes[0].set_yticks(range(len(top_publishers)))
axes[0].set_yticklabels(top_publishers.index, fontsize=9)
axes[0].set_xlabel('Number of Articles')
axes[0].set_title('Top 15 Publishers by Article Count')
axes[0].invert_yaxis()
axes[0].grid(True, alpha=0.3, axis='x')

# Add value labels
for i, v in enumerate(top_publishers.values):
    axes[0].text(v + max(top_publishers.values) * 0.01, i, f'{v:,}', 
                va='center', fontsize=8)

# Distribution of articles per publisher (log scale)
axes[1].hist(publisher_counts.values, bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[1].set_xlabel('Number of Articles per Publisher')
axes[1].set_ylabel('Number of Publishers')
axes[1].set_title('Distribution of Articles per Publisher')
axes[1].set_yscale('log')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'publisher_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Publisher analysis complete!")


### 3.3 Publication Date Trends


In [ ]:
# Analyze publication dates
print("=== Publication Date Analysis ===")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Total days covered: {(df['date'].max() - df['date'].min()).days} days")

# Articles by year
yearly_counts = df['year'].value_counts().sort_index()
print("\nArticles by Year:")
display(yearly_counts)

# Articles by month
monthly_counts = df['month'].value_counts().sort_index()
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
print("\nArticles by Month:")
monthly_df = pd.DataFrame({
    'Month': [month_names[m-1] for m in monthly_counts.index],
    'Count': monthly_counts.values
})
display(monthly_df)

# Articles by day of week
dow_counts = df['day_of_week'].value_counts()
dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_counts = dow_counts.reindex([d for d in dow_order if d in dow_counts.index])
print("\nArticles by Day of Week:")
display(dow_counts)

# Articles by hour
hourly_counts = df['hour'].value_counts().sort_index()
print("\nTop 10 Hours with Most Articles:")
display(hourly_counts.head(10))


In [ ]:
# Time series visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Daily article count over time
daily_counts = df.groupby(df['date'].dt.date).size()
axes[0, 0].plot(daily_counts.index, daily_counts.values, linewidth=1.5, color='steelblue')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Number of Articles')
axes[0, 0].set_title('Daily Article Publication Frequency Over Time')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].tick_params(axis='x', rotation=45)

# Articles by month
axes[0, 1].bar(range(len(monthly_counts)), monthly_counts.values, color='coral')
axes[0, 1].set_xticks(range(len(monthly_counts)))
axes[0, 1].set_xticklabels([month_names[m-1] for m in monthly_counts.index], rotation=45)
axes[0, 1].set_ylabel('Number of Articles')
axes[0, 1].set_title('Articles by Month')
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Articles by day of week
axes[1, 0].bar(range(len(dow_counts)), dow_counts.values, color='mediumseagreen')
axes[1, 0].set_xticks(range(len(dow_counts)))
axes[1, 0].set_xticklabels(dow_counts.index, rotation=45)
axes[1, 0].set_ylabel('Number of Articles')
axes[1, 0].set_title('Articles by Day of Week')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Articles by hour of day
axes[1, 1].bar(hourly_counts.index, hourly_counts.values, color='gold', alpha=0.7)
axes[1, 1].set_xlabel('Hour of Day (UTC)')
axes[1, 1].set_ylabel('Number of Articles')
axes[1, 1].set_title('Articles by Hour of Day')
axes[1, 1].grid(True, alpha=0.3, axis='y')
axes[1, 1].set_xticks(range(0, 24, 2))

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'publication_time_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Publication date analysis complete!")


## 4. Text Analysis and Topic Modeling


In [ ]:
# Initialize NLP tools
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    """Clean and preprocess text for analysis"""
    if pd.isna(text):
        return []
    
    # Convert to lowercase
    text = str(text).lower()
    
    # Remove special characters but keep spaces
    text = re.sub(r'[^a-z\s]', ' ', text)
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords and short words
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    
    # Lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return tokens

print("Text preprocessing function ready!")


In [ ]:
# Process all headlines
print("Processing headlines for text analysis...")
df['processed_tokens'] = df['headline'].apply(preprocess_text)

# Flatten all tokens
all_tokens = []
for tokens in df['processed_tokens']:
    all_tokens.extend(tokens)

# Count word frequencies
word_freq = Counter(all_tokens)
top_words = word_freq.most_common(50)

print(f"\nTotal unique words: {len(word_freq)}")
print(f"Total word occurrences: {len(all_tokens):,}")
print(f"\nTop 50 Most Common Words:")
for word, count in top_words:
    print(f"  {word}: {count:,}")


In [ ]:
# Visualize top words
top_30_words = top_words[:30]
words, counts = zip(*top_30_words)

fig, ax = plt.subplots(figsize=(12, 8))
ax.barh(range(len(words)), counts, color='steelblue')
ax.set_yticks(range(len(words)))
ax.set_yticklabels(words)
ax.set_xlabel('Frequency')
ax.set_title('Top 30 Most Common Words in Headlines')
ax.invert_yaxis()
ax.grid(True, alpha=0.3, axis='x')

# Add value labels
for i, v in enumerate(counts):
    ax.text(v + max(counts) * 0.01, i, f'{v:,}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'top_words_analysis.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Identify financial keywords and phrases
financial_keywords = [
    'price', 'target', 'stock', 'share', 'earnings', 'revenue', 'profit',
    'fda', 'approval', 'merger', 'acquisition', 'dividend', 'ipo',
    'analyst', 'rating', 'upgrade', 'downgrade', 'forecast', 'guidance',
    'quarter', 'annual', 'report', 'beat', 'miss', 'expectation'
]

print("=== Financial Keywords Analysis ===")
keyword_counts = {}
for keyword in financial_keywords:
    count = sum(1 for tokens in df['processed_tokens'] if keyword in tokens)
    keyword_counts[keyword] = count

keyword_df = pd.DataFrame(list(keyword_counts.items()), 
                         columns=['Keyword', 'Frequency'])
keyword_df = keyword_df.sort_values('Frequency', ascending=False)

print(f"\nKeyword frequencies:")
display(keyword_df)

# Visualization
fig, ax = plt.subplots(figsize=(10, 8))
ax.barh(range(len(keyword_df)), keyword_df['Frequency'], color='coral')
ax.set_yticks(range(len(keyword_df)))
ax.set_yticklabels(keyword_df['Keyword'])
ax.set_xlabel('Frequency')
ax.set_title('Financial Keywords Frequency in Headlines')
ax.invert_yaxis()
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'financial_keywords.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Text analysis complete!")


## 5. Time Series Analysis - Publication Frequency


In [ ]:
# Resample to different time frequencies
df_time = df.set_index('date')

# Daily frequency
daily_freq = df_time.resample('D').size()

# Weekly frequency
weekly_freq = df_time.resample('W').size()

# Monthly frequency
monthly_freq = df_time.resample('M').size()

print("=== Publication Frequency Statistics ===")
print(f"\nDaily Statistics:")
print(f"  Mean articles per day: {daily_freq.mean():.2f}")
print(f"  Median articles per day: {daily_freq.median():.2f}")
print(f"  Max articles in a day: {daily_freq.max()}")
print(f"  Min articles in a day: {daily_freq.min()}")
print(f"  Std deviation: {daily_freq.std():.2f}")

print(f"\nWeekly Statistics:")
print(f"  Mean articles per week: {weekly_freq.mean():.2f}")
print(f"  Median articles per week: {weekly_freq.median():.2f}")
print(f"  Max articles in a week: {weekly_freq.max()}")

print(f"\nMonthly Statistics:")
print(f"  Mean articles per month: {monthly_freq.mean():.2f}")
print(f"  Median articles per month: {monthly_freq.median():.2f}")
print(f"  Max articles in a month: {monthly_freq.max()}")


In [ ]:
# Identify spikes in publication frequency
daily_freq_df = daily_freq.reset_index()
daily_freq_df.columns = ['date', 'count']

# Calculate z-scores to identify outliers
mean_daily = daily_freq.mean()
std_daily = daily_freq.std()
daily_freq_df['z_score'] = (daily_freq_df['count'] - mean_daily) / std_daily

# Days with unusually high publication (z-score > 2)
spike_days = daily_freq_df[daily_freq_df['z_score'] > 2].sort_values('count', ascending=False)

print("=== Days with Unusually High Publication Frequency (Z-score > 2) ===")
print(f"Number of spike days: {len(spike_days)}")
if len(spike_days) > 0:
    print("\nTop 10 spike days:")
    display(spike_days.head(10)[['date', 'count', 'z_score']])

# Visualize time series with spikes highlighted
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Daily frequency plot
axes[0].plot(daily_freq.index, daily_freq.values, linewidth=1, color='steelblue', alpha=0.7)
axes[0].axhline(mean_daily, color='red', linestyle='--', label=f'Mean: {mean_daily:.1f}')
axes[0].axhline(mean_daily + 2*std_daily, color='orange', linestyle='--', 
                label=f'Mean + 2σ: {mean_daily + 2*std_daily:.1f}')
if len(spike_days) > 0:
    spike_dates = spike_days['date'].values
    spike_counts = spike_days['count'].values
    axes[0].scatter(spike_dates, spike_counts, color='red', s=50, zorder=5, 
                    label='Spikes (z>2)')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Number of Articles')
axes[0].set_title('Daily Publication Frequency Over Time (with Spike Detection)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# Weekly frequency plot
axes[1].plot(weekly_freq.index, weekly_freq.values, linewidth=2, color='coral', marker='o', markersize=3)
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Number of Articles')
axes[1].set_title('Weekly Publication Frequency Over Time')
axes[1].grid(True, alpha=0.3)
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'publication_frequency_timeseries.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Time series analysis complete!")


## 6. Publisher Analysis


In [ ]:
# Analyze publisher domains (if publisher contains email addresses)
def extract_domain(publisher):
    """Extract domain from publisher field (if it's an email)"""
    if pd.isna(publisher):
        return None
    publisher_str = str(publisher).lower()
    if '@' in publisher_str:
        return publisher_str.split('@')[1]
    return publisher_str

df['publisher_domain'] = df['publisher'].apply(extract_domain)

# Count by domain
domain_counts = df['publisher_domain'].value_counts()

print("=== Publisher Domain Analysis ===")
print(f"Total unique domains/publishers: {len(domain_counts)}")
print(f"\nTop 20 Domains/Publishers:")
display(domain_counts.head(20))

# Calculate concentration metrics
top_10_pct = (domain_counts.head(10).sum() / len(df) * 100).round(2)
top_20_pct = (domain_counts.head(20).sum() / len(df) * 100).round(2)

print(f"\nConcentration Metrics:")
print(f"  Top 10 publishers account for {top_10_pct}% of articles")
print(f"  Top 20 publishers account for {top_20_pct}% of articles")


In [ ]:
# Analyze publisher activity over time
top_publishers_list = publisher_counts.head(10).index.tolist()

# Monthly activity by top publishers
publisher_monthly = df[df['publisher'].isin(top_publishers_list)].groupby(
    ['publisher', 'year', 'month']
).size().reset_index(name='count')

# Create visualization
fig, ax = plt.subplots(figsize=(16, 8))

for publisher in top_publishers_list[:5]:  # Top 5 for clarity
    pub_data = publisher_monthly[publisher_monthly['publisher'] == publisher]
    pub_data['date'] = pd.to_datetime(
        pub_data[['year', 'month']].assign(day=1)
    )
    ax.plot(pub_data['date'], pub_data['count'], marker='o', label=publisher, linewidth=2)

ax.set_xlabel('Date')
ax.set_ylabel('Number of Articles')
ax.set_title('Monthly Publication Activity of Top 5 Publishers Over Time')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'publisher_activity_timeseries.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Publisher analysis complete!")


## 7. Stock Symbol Analysis


In [ ]:
# Analyze stock symbols
if 'stock' in df.columns:
    stock_counts = df['stock'].value_counts()
    
    print("=== Stock Symbol Analysis ===")
    print(f"Total unique stocks: {df['stock'].nunique()}")
    print(f"\nTop 20 Most Mentioned Stocks:")
    display(stock_counts.head(20))
    
    # Visualization
    top_20_stocks = stock_counts.head(20)
    
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.barh(range(len(top_20_stocks)), top_20_stocks.values, color='mediumseagreen')
    ax.set_yticks(range(len(top_20_stocks)))
    ax.set_yticklabels(top_20_stocks.index)
    ax.set_xlabel('Number of Articles')
    ax.set_title('Top 20 Stocks by Article Count')
    ax.invert_yaxis()
    ax.grid(True, alpha=0.3, axis='x')
    
    # Add value labels
    for i, v in enumerate(top_20_stocks.values):
        ax.text(v + max(top_20_stocks.values) * 0.01, i, f'{v:,}', 
                va='center', fontsize=9)
    
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'stock_symbol_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✅ Stock symbol analysis complete!")
else:
    print("⚠️  'stock' column not found in dataset")


## 8. Summary and Key Insights


In [ ]:
print("=" * 60)
print("EDA SUMMARY - KEY INSIGHTS")
print("=" * 60)

print(f"\n📊 Dataset Overview:")
print(f"  • Total articles: {len(df):,}")
print(f"  • Date range: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"  • Unique publishers: {df['publisher'].nunique()}")
if 'stock' in df.columns:
    print(f"  • Unique stocks: {df['stock'].nunique()}")

print(f"\n📝 Headline Statistics:")
print(f"  • Average headline length: {df['headline_length'].mean():.1f} characters")
print(f"  • Average word count: {df['headline_word_count'].mean():.1f} words")

print(f"\n📰 Publisher Insights:")
print(f"  • Top publisher: {publisher_counts.index[0]} ({publisher_counts.iloc[0]:,} articles)")
print(f"  • Top 10 publishers account for {top_10_pct}% of all articles")

print(f"\n⏰ Temporal Patterns:")
print(f"  • Most active day of week: {dow_counts.index[0]} ({dow_counts.iloc[0]:,} articles)")
print(f"  • Most active hour: {hourly_counts.index[0]}:00 UTC ({hourly_counts.iloc[0]:,} articles)")
print(f"  • Average articles per day: {daily_freq.mean():.2f}")

print(f"\n🔤 Text Analysis:")
print(f"  • Total unique words: {len(word_freq):,}")
print(f"  • Most common word: '{top_words[0][0]}' ({top_words[0][1]:,} occurrences)")

if len(spike_days) > 0:
    print(f"\n📈 Publication Spikes:")
    print(f"  • Days with unusually high publication: {len(spike_days)}")
    print(f"  • Highest spike: {spike_days.iloc[0]['date'].date()} ({spike_days.iloc[0]['count']} articles)")

print("\n" + "=" * 60)
print("✅ EDA Analysis Complete!")
print("=" * 60)
